
# Script to run the ML algorithms on the reduced dimensions generated by PCA for STUDY1 and test on STUDY2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')
import logging
LOG_FILENAME = 'UX_Ratings_PCA.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.INFO)
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import KFold,GridSearchCV,RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from RegscorePy.aic import aic # for calculating Akaike’s Information Criterion
from RegscorePy.bic import bic # for calculating Bayesian Information Criterion

In [ ]:
def readDataFromCsv(file):
    import pandas as pd
    print ("Reading the file from: ",file)
    df = pd.read_csv(file)
    return df

def loadDataset(data='study1',path='../datasets/files_generated/UX/study1_features_data.csv',target='PQ',app='Spell'):    
#     if data== 'study1':
    df = readDataFromCsv(path)
#     if data=='study2':
#         df = readDataFromCsv(path)
    df=df[df['App']==app]   
    print('The shape of the data  currently: ',df.shape)
    
    ## This should not have been there
    if(df.isnull().values.any()==True and data=='study1'):
        df = df.dropna()
        print('The shape of the data after dropping null values: ',df.shape)
    if data == 'startData':
#         df_join= pd.merge(df_stat_summ_withoutna, df_ux, on=['user_id','App','Cond','sessionNr'])
        X,y= df.drop(['PQ', 'ATT', 'HQI', 'HQS', 'HQ'],axis=1),df[target]
    elif data=='study1':
#         df_join= pd.merge(df_stat_summ_withoutna, df_ux, on=['user_id','App','Cond','sessionNr'])
        X,y= df.drop(['user_id','App','Cond','sessionNr','SEA', 'PQ', 'ATT', 'HQI', 'HQS', 'HQ'],axis=1),df[target]
    elif data=='study2':
#         df_join= pd.merge(df_stat_summ_withoutna, df_ux, how='inner',left_on=['user_id','Cond','sessionNr'],
#                            right_on=['UserId','IconSize','Session'])
        X,y=df.drop(['sessionNr','App','user_id','Size','UserId', 'Session', 
                     'PQ', 'ATT', 'HQI', 'HQS', 'HQ', 'IconSize'],axis=1),df[target]
        print(X.shape)
#     print('shape after join: ',df_join.shape)
    df_result={'data':X,'target':y}
    return df_result

In [ ]:
# path=['../datasets/files_generated/UX/study2_features_data_out_mahalanobis.csv']
# target='PQ'
# data='study2'
# dd = loadDataset(path=path[0],target=target,data=data)
# X = dd.get('data')
# y = dd.get('target')


# print("shape:",X.shape)

# # remove 0 variance
# col_index = np.where(X.var()!=0)
# columns = X.loc[:, X.var()== 0.0].columns.values
# X = X.loc[:, X.var() != 0.0]

# not_columns=['SEA','PQ','ATT', 'HQI', 'HQ','HQS']
# if data =='study1':
#     normality_test_features_path= 'Tables/NormalityCheck/study1_univariate_normality_test_features_mahalanobis_transformed.csv'
# else:
#     normality_test_features_path ='Tables/NormalityCheck/study2_univariate_normality_test_features_mahalanobis_transformed.csv'
# print("reading the normal features from path: ",normality_test_features_path)
# mahalanobis = pd.read_csv(normality_test_features_path)
# mahalanobis = list(mahalanobis[mahalanobis['Normality']==True]['Features'].values)
# for col in not_columns:
#     if(col in mahalanobis):
#         mahalanobis.remove(col)

# # X=X[mahalanobis]

# # Create correlation matrix
# corr_matrix = X.corr().abs()

# # Select upper triangle of correlation matrix
# upper_traingle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.95
# to_drop_cols = [column for column in upper_traingle.columns if any(upper_traingle[column] >= 0.80)]

# # Drop features 
# X = X.drop(X[to_drop_cols], axis=1)
# print("current shape:",X.shape)

In [ ]:
# normality_test_features_path= 'Tables/NormalityCheck/study1_univariate_normality_test_features_mahalanobis_transformed.csv'
# mahalanobis_study1 = pd.read_csv(normality_test_features_path)
# mahalanobis_study1 = list(mahalanobis_study1[mahalanobis_study1['Normality']==True]['Features'].values)
# not_columns=['SEA','PQ','ATT', 'HQI', 'HQ','HQS']
# for col in not_columns:
#     if(col in mahalanobis_study1):
#         mahalanobis_study1.remove(col)

# normality_test_features_path= 'Tables/NormalityCheck/study2_univariate_normality_test_features_mahalanobis_transformed.csv'
# mahalanobis_study2 = pd.read_csv(normality_test_features_path)
# mahalanobis_study2 = list(mahalanobis_study2[mahalanobis_study2['Normality']==True]['Features'].values)
# not_columns=['SEA','PQ','ATT', 'HQI', 'HQ','HQS']
# for col in not_columns:
#     if(col in mahalanobis_study2):
#         mahalanobis_study2.remove(col)
        
# set1=set(mahalanobis_study1)
# set2=set(mahalanobis_study2)

# missing = list(sorted(set1 - set2))
# added = list(sorted(set2 - set1))

# print('missing:', missing)
# print('added:', added)
# print(len(mahalanobis_study1))
# print(len(mahalanobis_study2))

In [ ]:
def optimalModelSelection(model,param_grid,X,y,method='grid'):
    '''Tune the hyperparameters to find the best score personality data'''
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import StandardScaler,RobustScaler
    from sklearn.pipeline import make_pipeline,Pipeline
    from sklearn.model_selection import KFold,GridSearchCV,RandomizedSearchCV
    from sklearn.decomposition import PCA
    
#     K = 10
#     kf = KFold(n_splits=K, shuffle=True,random_state=32)
    
    scoring={'r2':'r2','mse':'neg_mean_squared_error','mae':'neg_mean_absolute_error'}
#     pca = PCA(random_state=32,svd_solver='full')
#     pipe = Pipeline([('scl', StandardScaler()),
#                      ('pca',pca),
#                     ('clf', model)])
    if(method=='grid'):
        search = GridSearchCV(model, param_grid, cv=10,n_jobs=-1,scoring=scoring,return_train_score=True,refit='r2')
        search.fit(X,y)
    if(method=='random'):
        search=RandomizedSearchCV(estimator = model, param_distributions = param_grid, 
                               n_iter = 100, cv = 10, verbose=1, 
                               random_state=32, n_jobs = -1,scoring=scoring,return_train_score=True,refit='r2')
        search.fit(X,y)
    
#     logging.info('Reduced dimension for 98% variance: {}'.format(search.best_estimator_.named_steps['pca'].components_.shape))
#     logging.info('Reduced dimension: {}'.format(search.best_estimator_.named_steps['pca'].n_components_))
    
    print('Best params: {}'.format(search.best_params_))
    logging.info('Best params: {}'.format(search.best_params_))
#     print('Best score after fitting the estimator with best params:{}'.format(search.best_score_))
#     logging.info('Best score after fitting the estimator with best params:{}'.format(search.best_score_))
    print('RMSE: %0.2f'%(np.sqrt(-search.cv_results_['mean_test_mse'][search.best_index_])))
    print("R2(Validation): %0.2f (+/- %0.2f)" % (search.best_score_,search.cv_results_['std_test_r2'][search.best_index_]))
    print("R2(Train): %0.2f (+/- %0.2f)" % (search.cv_results_['mean_train_r2'][search.best_index_],
                                                 search.cv_results_['std_train_r2'][search.best_index_]))
    print("MAE(Validation): %0.2f (+/- %0.2f)" % (-search.cv_results_['mean_test_mae'][search.best_index_],
                                                  search.cv_results_['std_test_mae'][search.best_index_]))
    print("MAE(Train): %0.2f (+/- %0.2f)" % (-search.cv_results_['mean_train_mae'][search.best_index_],
                                                 search.cv_results_['std_train_mae'][search.best_index_]))
    
    logging.info('RMSE: %0.2f'%(np.sqrt(-search.cv_results_['mean_test_mse'][search.best_index_])))
    logging.info("R2: %0.2f (+/- %0.2f)" % (search.best_score_,search.cv_results_['std_test_r2'][search.best_index_]))
    return search.best_estimator_,search.best_params_, search.best_score_,search.cv_results_,search.best_index_

In [ ]:
def perform_evaluation(pathsArr,model,param_grid,method='grid',data='study1',transformation=False,n_components=0.95):
    
    if transformation==False:
#         pathsArr=['../datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv']
        targets=['PQ', 'ATT']
        #keys=['none','mahalanobis','manhattan']
        keys=['mahalanobis']
    else:
#         pathsArr=['../datasets/files_generated/UX/study1_features_data_out_mahalanobis_transformedDistributions.csv']
        ##remove the non-normal distributed features
        not_columns=['SEA','PQ','ATT', 'HQI', 'HQ','HQS']
        if data =='study1':
#             normality_test_features_path= '../../Tables/NormalityCheck/study1_univariate_normality_test_features_mahalanobis_transformed.csv'
            normality_test_features_path ='/mnt/vdb1/UX-Ratings/NormalityCheck/study1_univariate_normality_test_features_mahalanobis_transformed.csv'
        else:
#             normality_test_features_path ='Tables/NormalityCheck/study2_univariate_normality_test_features_mahalanobis_transformed.csv'
            normality_test_features_path ='/mnt/vdb1/UX-Ratings/NormalityCheck/study2_univariate_normality_test_features_mahalanobis_transformed.csv'
        print("reading the normal features from path: ",normality_test_features_path)
        mahalanobis = pd.read_csv(normality_test_features_path)
        mahalanobis = list(mahalanobis[mahalanobis['Normality']==True]['Features'].values)

#         manhattan = pd.read_csv('Tables/Study1/study1_univariate_normality_test_features_manhattan_transformed.csv')
#         manhattan = list(manhattan[manhattan['Normality']==True]['Features'].values)
        
#         none_set = pd.read_csv('tables/Study1/study1_univariate_normality_test_features_none_transformed.csv')
#         none_set = list(none_set[none_set['Normality']==True]['Features'].values)
        
        # print(columns_set[0])
        for col in not_columns:
        #     print(col)
            if(col in mahalanobis):
                mahalanobis.remove(col)
#             if (col in manhattan): 
#                 manhattan.remove(col)
#             if (col in none_set): 
#                 none_set.remove(col)
        #columns_set=[none_set,mahalanobis,#manhattan]
#         columns_set=[mahalanobis]
        targets=['PQ', 'ATT']
        #keys=[#'none','mahalanobis',#'manhattan']
        keys=['mahalanobis_transformedDistributions']
    
    #store the results
    results_r2_val_scores={}
    results_r2_test_scores={}
    results_r2_train_scores={}
    results_rmse_test={}
    results_rmse_train={}
    results_rmse_val={}
    results_adjusted_r2_val_scores={}
    results_std_r2_val_scores={}
    results={}
    results_val_mae={}
    results_test_mae={}
    results_train_mae={}
    predictions={}
    results_aic_test={}
    results_bic_test={}
    results_aic_val={}
    results_bic_val={}
    results_mape_val={}
    results_mape_test={}
    for target in targets:
        std_val_scores={}
        val_scores={}
        test_scores ={}
        train_scores={}
        params_spell={}
        rmse_train={}
        rmse_test={}
        rmse_val={}
        mae_val={}
        mae_test={}
        mae_train={}
        adjusted_test={}
        bic_val ={}
        bic_test ={}
        aic_val={}
        aic_test={}
        mape_val={}
        mape_test={}
        logging.info('Prediction for {}'.format(target))
        print('Prediction for {}'.format(target))
        i=0
#         print(zip(pathsArr[0],keys))
        for path,key in zip([pathsArr[0]],keys):
            print(path)
            personality=loadDataset(data=data,path=path,target=target)
            X=personality.get('data')
            zero_var_columns = X.loc[:, X.var() == 0.0].columns
            print("columns thrown away because they have 0 variance:",zero_var_columns)
            
            # removed 0 variance
            X = X.loc[:, X.var() != 0.0]
           
            print(X.isnull().values.any())
            
            if transformation==True:
                X=X[mahalanobis]
                print("Shape of the data after selected transformed columns:",X.shape)
#                 i=i+1
            y=personality.get('target')
            
            # remove highly correlated data 
            
            # Create correlation matrix
            corr_matrix = X.select_dtypes(['float64']).corr().abs()

            # Select upper triangle of correlation matrix
            upper_traingle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

            # Find index of feature columns with correlation greater than 0.95
            to_drop_cols = [column for column in upper_traingle.columns if any(upper_traingle[column] >= 0.80)]
            print(to_drop_cols)
            # Drop features 
            X = X.drop(X[to_drop_cols], axis=1)
            
            print("Shape of the data after removing 0 variance highly correlated data:",X.shape)

#             # split the data into train test set
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
#             X_train =X
#             y_train = y
            
            test = loadDataset(data='study2',path=pathsArr[1],target=target)
            X_test = test.get('data')
            y_test = test.get('target')
            
            
            # Drop features to make both the datasets schema equal
            X_test = X_test.drop(X_test[zero_var_columns], axis=1)
            if transformation==True:
                X_test=X_test[mahalanobis]
            X_test = X_test.drop(X_test[to_drop_cols], axis=1)
            
            print("Shape of the data after removing 0 variance highly correlated data:",X_test.shape)
            print(np.equal(X_train.columns,X_test.columns))
            
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train=scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            print(np.where(np.isnan(X_train)==True))
            
            pca= PCA(n_components=n_components)
            pca.fit(X_train)
            print('number of principal components:',pca.n_components_)
            
            logging.info('number of principal components: {}'.format(pca.n_components_))
            predictors=pca.n_components_
            X_train = pca.transform(X_train)
            X_test = pca.transform(X_test)
            print("shape of training data ",X_train.shape)
            print("shape of test data",X_test.shape)
            logging.info("shape of training data {}".format(X_train.shape))
            logging.info("shape of test data {}".format(X_test.shape))
                
            estimator, best_params_,best_score_,cv_results_,best_index_ = optimalModelSelection(
                    model,param_grid,X_train,y_train,method)
            
            # calculate the RSS on test set
            #rss_val = (np.array(y_train)-estimator.predict(X_train)).sum()
#             print("RSS(Validation): %0.2f" %(rss_val))
            print('Performance(R2):%0.2f'%(best_score_))
            # calculate the AIC
            y_pred_train  = estimator.fit(X_train,y_train).predict(X_train)
            aic_score_val = aic(y_train,y_pred_train,X_train.shape[1])
            # calculate Bayesian Information Criterion
            bic_score_val = bic(y_train,y_pred_train,X_train.shape[1])
            mape_score_val = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100
            
            # test on unseen data
            y_pred = estimator.predict(X_test)
            aic_score_test = aic(y_test,y_pred,X_test.shape[1])
            bic_score_test = bic(y_test,y_pred,X_test.shape[1])
            mape_score_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
            
            '''TODO: Store the residuals in the table'''
            residuals_test = np.array(y_test)- y_pred
#             rss_test = np.linalg.norm(np.array(y_test)-estimator.predict(X_test))**2
            #print(len(residuals))
            
            #store it in a seperate table
            residuals = np.array(y_train)- y_pred_train
#             prediction= {'Original':np.array(y_train),'Predicted':y_pred_train,'Residuals':residuals}
            prediction= {'Original':np.array(y_test),'Predicted':y_pred,'Residuals':residuals_test}
            predictions[target]=prediction
            
            score = r2_score(y_test,y_pred)
            rmse = np.sqrt(np.abs(mean_squared_error(y_test,y_pred)))
            mae = mean_absolute_error(y_test,y_pred)
            print('Peforming predictions on unseen data')
            print('Performance(R2):%0.2f | RMSE:%0.2f | MAE:%0.2f | AIC:%0.2f | MAPE:%0.2f' %(score,rmse,mae,aic_score_test,mape_score_test))
            
            #append the result
            
            # RSquared score
            val_scores[key]=best_score_
            train_scores[key]=cv_results_['mean_train_r2'][best_index_]
            params_spell[key]=best_params_
            test_scores[key]=score
            
            # RMSE 
            rmse_train[key]=np.sqrt(np.abs(cv_results_['mean_train_mse'][best_index_]))
            std_val_scores[key]= cv_results_['std_test_r2'][best_index_]
            rmse_val[key]= np.sqrt(np.abs(cv_results_['mean_test_mse'][best_index_]))
            rmse_test[key]=rmse
            
            # Adjusted RSquared
            adjusted_test[key]=1 - (1-best_score_)*(len(y_train)-1)/(len(y_train)-predictors-1)
            
            # MAE 
            mae_train[key]= -cv_results_['mean_train_mae'][best_index_]
            mae_val[key]= -cv_results_['mean_test_mae'][best_index_]
            mae_test[key]=mae
            
            #AIC / BIC 
            aic_val[key]=aic_score_val
            aic_test[key]= aic_score_test
            bic_val[key]=bic_score_val
            bic_test[key]=bic_score_test
            
            #MAPE
            mape_val[key]=mape_score_val
            mape_test[key]=mape_score_test
            
            print('*'*100)
            logging.info('*'*100)
                
        results_r2_val_scores[target]=val_scores
        results_r2_train_scores[target]=train_scores
        results_rmse_val[target]=rmse_val
        results_rmse_train[target]=rmse_train
        results_adjusted_r2_val_scores[target]=adjusted_test
        results_rmse_test[target]=rmse_test
        results_r2_test_scores[target]=test_scores
        results_std_r2_val_scores[target]=std_val_scores
        results_val_mae[target]=mae_val
        results_train_mae[target]=mae_train
        results_test_mae[target]=mae_test
        results_aic_test[target]=aic_test
        results_aic_val[target]=aic_val
        results_bic_test[target]=bic_test
        results_bic_val[target]=bic_val
        results_mape_val[target]=mape_val
        results_mape_test[target]=mape_test
    
    results['r2_train']=results_r2_train_scores
    results['r2_validation']=results_r2_val_scores
    results['rmse_train']=results_rmse_test
    results['rmse_validation']=results_rmse_val
    results['rmse_test']=results_rmse_test
    results['r2_test']=results_r2_test_scores
    results['std_validation']= results_r2_test_scores
    results['std_validation']= results_std_r2_val_scores
    results['adjusted_r2_val']=results_adjusted_r2_val_scores
    results['mae_train']=results_train_mae
    results['mae_validation']=results_val_mae
    results['mae_test']=results_test_mae
    results['mae_test']=results_test_mae
    results['aic_test']=results_aic_test
    results['aic_validation']=results_aic_val
    results['bic_test']=results_bic_test
    results['bic_validation']=results_bic_val
    results['mape_validation']=results_mape_val
    results['mape_test']=results_mape_test
        
    print('*'*100)
    logging.info('*'*100)
    return results, predictions

In [ ]:
def runAllModels(pathsArr, filename,data='study1',transformation=False,n_components=0.95):
    # #create models
    
    np.random.seed(32)
    #linear regression
    logging.info('********Applying Linear Regression****************')
    from sklearn.linear_model import LinearRegression
#     fit_intercept_space=[True,False]
#     normalize_space=[True,False]
    fit_intercept_space=[True]
    param_grid={'fit_intercept':fit_intercept_space,
#                 'normalize':normalize_space
               }
    regr= LinearRegression()
    results_regr,predictions_regr= perform_evaluation(pathsArr,regr,param_grid,data=data,transformation=transformation,n_components=n_components)
#     print(results_regr)

    ## lasso regression
    logging.info('********Applying Lasso Regression****************')
    from sklearn.linear_model import Lasso
    alpha_space = np.logspace(0, 1, 100)
#     alpha_space = [0.001,0.01,0.1,1.0,2,3]
    param_grid={'alpha':alpha_space}
    lasso = Lasso(random_state=32)
    results_lasso,predictions_lasso = perform_evaluation(pathsArr,lasso,param_grid,data=data,
                                                        transformation=transformation,n_components=n_components)
    


    ## elastic net 
    logging.info('********Applying Elastic Net Regression****************')
    from sklearn.linear_model import ElasticNet
    alpha_space = np.logspace(0, 2 , 50)
    param_grid={'alpha':alpha_space}
    enet = ElasticNet(random_state=32)
    results_enet,predictions_enet = perform_evaluation(pathsArr,enet,param_grid,data=data,transformation=transformation,n_components=n_components)

    # ##create models
    np.random.seed(19)
    # support vector machines
    logging.info('********Applying Support vector machine****************')
    from sklearn.svm import SVR
    C_space=np.logspace(-1,1,10)
#     C_space = np.logspace(-2, 10, 5)
    epsilon_space= np.logspace(-1,0,10)
#     epsilon_space=np.logspace(-1,2,5)
    gamma_space = np.logspace(-3, -2, 10)
#     gamma_space='scale'
    param_grid={'C':C_space,'epsilon':epsilon_space,'gamma':gamma_space}
    svr = SVR(kernel = 'rbf')
    results_svm,predictions_svm = perform_evaluation(pathsArr,svr,param_grid,method='random',data=data,transformation=transformation,n_components=n_components)


#     # random forest
#     logging.info('********Applying Random Forest****************')
    from sklearn.ensemble import RandomForestRegressor
#     n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# #     n_estimators=[500,1000]
#     max_depth = [int(x) for x in np.linspace(10, 20, num = 11)]
#     min_samples_split = [2, 5, 10,15]
#     min_samples_leaf = [5,10,50,100,200]
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

    max_depth = [int(x) for x in np.linspace(1, 5, num = 5)]

    min_samples_split = [int(x) for x in np.linspace(10, 100, num = 10)]
    min_samples_leaf = [int(x) for x in np.linspace(10, 60, num = 20)]
    bootstrap = [True, False]
#     max_features=['sqrt']
    max_features=['auto','sqrt']
    param_grid={'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
#                     'bootstrap': bootstrap,
                    'max_features':max_features
                   }   
    rf = RandomForestRegressor(random_state=32)
    results_rf,predictions_rf= perform_evaluation(pathsArr,rf,param_grid,method='random',data=data,transformation=transformation,n_components=n_components)

    #Linear regression using stochastic gradient descent
    logging.info('********Applying Linear regression with stochastic gradient descent****************')
    from sklearn.linear_model import SGDRegressor
    param_grid={#'max_iter':[100,500,1000],
                'max_iter':[50,100],
                'penalty':[None],
                'eta0':[0.01,0.1,0.5]
                   }
    sgd_reg = SGDRegressor(random_state=32)
    results_sgd,predictions_sgd = perform_evaluation(pathsArr,sgd_reg,param_grid,data=data,transformation=transformation,n_components=n_components)
#     results_sgd

#     # MARS
    np.random.seed(20)
    logging.info('********Applying MARS****************')
    from pyearth import Earth
    max_degree_space=[1]
    #penalty_space=[3.0,6.0]
   # minspan_alpha = np.linspace(0, 0.5, num = 10)
    penalty_space=np.logspace(-1,1,20)
    minspan_alpha=np.logspace(-3,1,20)
    max_terms=[10,20,25]
#     max_terms=np.linspace(25,30,num=5)
    endspan_alpha = [0.05]
    # # endspan_alpha= np.linspace(0, 1.0, num = 10)
    # # endspan=[5]
    param_grid={'max_degree':max_degree_space,
        'penalty':penalty_space,
             #  'minspan_alpha':minspan_alpha,
        #'endspan_alpha':endspan_alpha,
                'use_fast':[True],
        'max_terms':max_terms
               }
    mars= Earth()
    results_mars,predictions_mars= perform_evaluation(pathsArr,mars,param_grid,method='grid',data=data,transformation=transformation,n_components=n_components)
    results_mars
    
    def createTable(results,name):
        '''Creates the final table'''
        df= pd.concat([pd.DataFrame(results.get('r2_test')).T,
        pd.DataFrame(results.get('r2_train')).T,
                  pd.DataFrame(results.get('r2_validation')).T,
        pd.DataFrame(results.get('rmse_test')).T,
                  pd.DataFrame(results.get('rmse_train')).T,
        pd.DataFrame(results.get('rmse_validation')).T,
                  pd.DataFrame(results.get('std_validation')).T,
                     pd.DataFrame(results.get('adjusted_r2_val')).T,
                       pd.DataFrame(results.get('mae_train')).T,
                      pd.DataFrame(results.get('mae_validation')).T,
                      pd.DataFrame(results.get('mae_test')).T,
                      pd.DataFrame(results.get('aic_validation')).T,
                      pd.DataFrame(results.get('aic_test')).T,
                      pd.DataFrame(results.get('bic_validation')).T,
                      pd.DataFrame(results.get('bic_test')).T,
                      pd.DataFrame(results.get('mape_validation')).T,
                      pd.DataFrame(results.get('mape_test')).T],axis=1)
        
        df.columns=['R2(Test)','R2(Train)','R2(Validation)',
                    'RMSE(Test)','RMSE(Train)','RMSE(Validation)',
                    'StandardError(Validation)',
                    'Adjusted R2(Validation)',
                    'MAE(Train)','MAE(Validation)','MAE(Test)',
                    'AIC(Validation)','AIC(Test)','BIC(Validation)','BIC(Test)',
                   'MAPE(Validation)','MAPE(Test)']
        df['Target']=df.index
        df['Algorithm']=name
        df= df.reset_index(drop=True)
        return df

    df_sgd=createTable(results_sgd,name='Linear Regression SGD')
    df_lasso=createTable(results_lasso,name='Lasso Regression')
    df_enet=createTable(results_enet,name='Elastic Net')
    df_svm=createTable(results_svm,name='SVM')
    df_rf=createTable(results_rf,name='Random Forest')
    df_mars=createTable(results_mars,name='MARS')
    df_lr=createTable(results_regr,name='Linear Regression')
    
    pd.concat([
        df_rf,df_svm,df_sgd,
        df_lr,
        df_lasso,
        df_enet,df_mars
    ]).to_csv(filename,index=False)
    
#     del df_sgd,df_lasso,df_enet,df_svm,df_rf,df_mars
    
    def createPredictionsTable(predictions):
        pq= pd.DataFrame(predictions.get('PQ'))
        pq.rename(index=str, columns={"Original": "Original_PQ", "Prediction": "Prediction_PQ",'Residuals':'Residuals_PQ'}, inplace=True)
        att=pd.DataFrame(predictions.get('ATT'))
        att.rename(index=str, columns={"Original": "Original_ATT", "Prediction": "Prediction_ATT",'Residuals':'Residuals_ATT'},inplace=True)
        df = pd.concat([pq,att],axis=1)
        return df
    
    df_sgd=createPredictionsTable(predictions_sgd)
    df_lasso=createPredictionsTable(predictions_lasso)
    df_enet=createPredictionsTable(predictions_enet)
    df_svm=createPredictionsTable(predictions_svm)
    df_rf=createPredictionsTable(predictions_rf)
    df_mars=createPredictionsTable(predictions_mars)
    df_lr=createPredictionsTable(predictions_regr)
    
#     print(df_lasso)
    if transformation==False:
        filename=str(data)+'_PCA_alltargets_mahalanobis_'+str(n_components)+'_predictions_unseen.xlsx'
    else:
        filename=str(data)+'_PCA_alltargets_mahalanobis_transformed_'+str(n_components)+'_predictions_unseen.xlsx'
        
    with pd.ExcelWriter(filename) as writer:  # doctest: +SKIP
        df_sgd.to_excel(writer, sheet_name='Linear Regression SGD')
        df_lasso.to_excel(writer, sheet_name='Lasso Regression')
        df_enet.to_excel(writer, sheet_name='Elastic Net')
        df_svm.to_excel(writer, sheet_name='SVM')
        df_rf.to_excel(writer, sheet_name='Random Forest')
        df_mars.to_excel(writer, sheet_name='MARS')
        df_lr.to_excel(writer, sheet_name='LR')
    
    print('File saved successfully')
    

In [ ]:
# #for checking the function
# # uncomment it when you want

# pathsArr=['../datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv']
# transformation=False
# n_components=0.95
# data='study1'
# from sklearn.linear_model import Lasso
# alpha_space = np.logspace(0, 1, 100)
# #     alpha_space = [0.001,0.01,0.1,1.0,2,3]
# param_grid={'alpha':alpha_space}
# lasso = Lasso(random_state=32)
# results_lasso,predictions_lasso = perform_evaluation(pathsArr,lasso,param_grid,data=data,
#                                                         transformation=transformation,n_components=n_components)


Study1 original and test on Study2 Original
---

In [ ]:
# evaluate model on study1 data and test on study2 on original distribution with 95% variance explained PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis.csv']
#     path = ['../../../datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv',
#          '../../../datasets/files_generated/UX/study2_features_data_out_mahalanobis.csv']
    filename='Tables/PCA_alltargets_mahalanobis_0.95PC.csv'
    runAllModels(path,filename,data='study1',transformation=False,n_components=0.95)

In [ ]:
# evaluate model on study1 data and test on study2 on original distribution with 80% variance explained PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis.csv']
    filename='Tables/PCA_alltargets_mahalanobis_0.80PC.csv'
    runAllModels(path,filename,data='study1',transformation=False,n_components=0.80)

In [ ]:
# evaluate model on study1 data and test on study2 on original distribution with 3 PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis.csv']
    filename='Tables/PCA_alltargets_mahalanobis_3PC.csv'
    runAllModels(path,filename,data='study1',transformation=False,n_components=3)

Study1 transformation and test on Study2 transformation
---

In [ ]:
# evaluate model on transformed distributions of study1 data and test on study2 with 95% variance explained PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis_transformedDistributions.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis_transformedDistributions.csv']
#     path=['../../../datasets/files_generated/UX/study1_features_data_out_mahalanobis_transformedDistributions.csv',
#          '../../../datasets/files_generated/UX/study2_features_data_out_mahalanobis_transformedDistributions.csv']
    filename='Tables/PCA_alltargets_mahalanobis_transformed_0.95PC.csv'
    runAllModels(path,filename,data='study1',transformation=True,n_components=0.95)

In [ ]:
# evaluate model on transformed distributions of study1 data and test on study2 with 80% variance explained PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis_transformedDistributions.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis_transformedDistributions.csv']
    filename='Tables/PCA_alltargets_mahalanobis_transformed_0.80PC.csv'
    runAllModels(path,filename,data='study1',transformation=True,n_components=0.80)

In [ ]:
# evaluate model on transformed distributions of study1 data and test on study2 with 3PC
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/UX/study1_features_data_out_mahalanobis_transformedDistributions.csv',
         '/mnt/vdb1/datasets/files_generated/UX/study2_features_data_out_mahalanobis_transformedDistributions.csv']
    filename='Tables/PCA_alltargets_mahalanobis_transformed_3PC.csv'
    runAllModels(path,filename,data='study1',transformation=True,n_components=3)